In [8]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, RobertaTokenizer
import json
from torch.utils.data import Dataset, DataLoader
import torch

In [6]:
# Load the tokenizer and model
MODEL_NAME = "Salesforce/codet5-small"
tokenizer = RobertaTokenizer.from_pretrained(MODEL_NAME, cache_dir="./cache")
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, cache_dir="./cache")

tokenizer_config.json:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/703k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/294k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/12.5k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [7]:
def load_data(ast_file, code_file, nl_file):
    with open(ast_file, "r") as f:
        ast_lines = f.readlines()
    with open(code_file, "r") as f:
        code_lines = f.readlines()
    with open(nl_file, "r") as f:
        nl_lines = f.readlines()
    
    data = []
    for ast, code, nl in zip(ast_lines, code_lines, nl_lines):
        input_text = f"Generate comment: AST: {ast.strip()} Code: {code.strip()}"
        target_text = nl.strip()
        data.append((input_text, target_text))
    
    return data

In [9]:
train_data = load_data("Data1\\data_round_1\\train\\train.token.ast", "Data1\\data_round_1\\train\\train.token.code", "Data1\\data_round_1\\train\\train.token.nl")

In [10]:
def tokenize_data(data, tokenizer, max_length=512):
    inputs, targets = zip(*data)
    input_encodings = tokenizer(list(inputs), padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    target_encodings = tokenizer(list(targets), padding=True, truncation=True, max_length=max_length, return_tensors="pt")
    return input_encodings, target_encodings

In [11]:
train_inputs, train_targets = tokenize_data(train_data, tokenizer)

In [12]:
class CodeDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        return {
            "input_ids": self.inputs["input_ids"][idx],
            "attention_mask": self.inputs["attention_mask"][idx],
            "labels": self.targets["input_ids"][idx]
        }

In [13]:
train_dataset = CodeDataset(train_inputs, train_targets)

In [14]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

In [15]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32100, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32100, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [22]:
EPOCHS = 3
BATCH_SIZE = 8
model.train()
for epoch in range(EPOCHS):
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    print(f"Epoch {epoch+1} Loss: {loss.item()}")

KeyboardInterrupt: 

In [ ]:
model.save_pretrained("codet5_finetuned")
tokenizer.save_pretrained("codet5_finetuned")